In [ ]:
try:
    import datatools_bdh
except ModuleNotFoundError:
    !pip install git+https://github.com/sfu-bigdata/datatools-bdh

from datatools_bdh.sampler import *
import matplotlib.pyplot as plt

## Rejection sampling of constrained domains

The constraints here are formed by a lower and upper bounds of the sample interval.
See code in `datatools_bdh.sampler.make_sample_bounded` for example usage.


In [ ]:


#print(", ".join([f"{x:.2f}" for x in make_normal_bounded(-.5, .5, nsamples=20)]))
lub = .5
nsamples = 100
#np.random.seed(432)
distribution = 'gaussian'
if distribution.lower() == 'gaussian':
    s = make_normal_bounded(-lub, +lub, nsamples=nsamples,
                            oversample=10,
                            sigma=lub*.3
                           )
elif distribution.lower() == 'power':
    s = make_power_bounded(-lub, +lub, nsamples=nsamples,
                            oversample=10,
                            a=.1
                           )
elif distribution.lower() == 'exponential':
    s = make_exponential_bounded(-lub, +lub, nsamples=nsamples,
                            oversample=10,
                            scale=.1
                           )
print(f"Generated {len(s)} samples from a constrained Gaussian distribution.")
plt.hist(s, bins=min(int(nsamples/2), 100));
plt.xlim([-lub, +lub]);

In [ ]:
# TODO
# - inverse transform sampling using inverse CDF
# - two-sided mirroring of one-sided, positive distribution domains

## Inverse transform (ITF) sampling

The following code uses the `ITFSampler` class to generate values from a distribution via inverse CDF transform sampling.

The simulated sample is shown via a bar chart of its binned histogram. The PDF of the desired distribution shown in the same plot, seems to agree well.

### How ITF sampling works
This method maps uniformly distributed values through the inverse CDF of the desired distribution.

A CDF, or cumulative distribution function, tells us what portion of values in a population is at or below a certain threshold level. For low threshold levels this function is usually close to 0 and for high values the CDF reaches a value of 1, meaning that the entire population of values is below the given level.

The inverse CDF gives us level values that are associated with probabilities that range from 0 to 1.

In [ ]:
from datatools_bdh.sampler import *
# works with commit https://github.com/sfu-bigdata/datatools-bdh/commit/8261828fe10abf442fe63e47923325452b0bd99a
import matplotlib.pyplot as plt
import scipy.stats as sps

In [ ]:
dist = sps.exponpow
b = 3
mean, var, skew, kurt = dist.stats(b, moments='mvsk')

smp = ITFSampler(distribution=dist, b=b)
nbins = int(smp.cdf_nsteps/10)

ys = smp(smp.x0)

In [ ]:
line_args = dict(lw=5, alpha=0.6)

#hist_ori = 'horizontal'
hist_ori = 'vertical'
fs = [sps.exponpow.cdf(smp.x, smp.b),
      ys.values]
fs_label = ['CDF', 'inverse CDF']
ypdf = sps.exponpow.pdf(smp.x, smp.b)

In [ ]:
if hist_ori == 'vertical':
    fs.append(ypdf)
    fs_label.append("PDF")

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(smp.x0, np.vstack(fs).T,
       '-', **line_args, label=fs_label)
if hist_ori == 'horizontal':
    ax.plot(ypdf,
            smp.x0, 
            '-', **line_args, label='flipped PDF')
ax.set_title('power exponential')
if True:
    plt.hist([#exponpow.cdf(x, b),
              ys],
             bins=nbins, density=True,
             color='peachpuff',
             orientation=hist_ori);
plt.legend();

In [ ]:
# further examples:

# distribute points in [0, 3] according to density of normal with sigma=1.5
# generate uniform y values using linspace
